# Decision tree algorithm from scratch in python


### Problem statement:

There are multiple types of balls in our data set as cricket ball and tennis ball. This type of balls is classified based on its weight and its surface. We must design application using machine learning strategy which is used to classify the balls.

### Dataset Description

- features -> Weight, Surface
- Label -> Tennis , Cricket

### Initial script

In [2]:
import pandas as pd
import numpy as np
from pprint import pprint
eps = np.finfo(float).eps 
import os
import pickle

‘eps’ here is the smallest representable number. At times we get log(0) or 0 in the denominator, to avoid that we are going to use this.

In [4]:
path = os.getcwd()

for i in range(3):
    path = os.path.dirname(path)

data = pd.read_csv( path + '/Datasets/Ball_Dataset.csv', names = ['Weight', 'Surface', 'Label'])

In [5]:
data

,Weight,Surface,Label
0,30,Rough,Tennis
1,31,Rough,Tennis
2,70,Smooth,Cricket
3,77,Smooth,Cricket
4,46,Rough,Tennis
5,78,Smooth,Cricket
6,47,Rough,Tennis
7,79,Smooth,Cricket
8,48,Rough,Tennis
9,49,Rough,Tennis


In [7]:
def entropy(target_col):

	"""Calculate the entropy of a dataset. The only parameter for this function is the target_col parameter which specifies the target column"""

	vals ,counts = np.unique(target_col,return_counts = True) 
	total = np.sum(counts)
	entropy = 0


	for i in counts:
		fraction = i/total
		entropy += -fraction * np.log2(fraction)

	entropy = round(entropy,3)
	return entropy

In [8]:
entropy(data['Label'])

0.981

In [9]:
def find_entropy_attribute(df, attribute):

	Target = df.keys()[-1]

	target_variables = df[Target].unique() # This gives all 'Yes' and 'No'

	variables = df[attribute].unique()  # this gives different features of attribute 

	attribute_entropy = 0

	for variable in variables:
		entropy = 0

		for target_variable in target_variables:

			num = len(df[attribute][df[attribute] == variable][df[Target] == target_variable])

			total = len(df[attribute][df[attribute] == variable])

			fraction = num / (total+eps)

			entropy += -fraction * np.log2(fraction + eps)

		fraction_1 = total/len(df)

		attribute_entropy += -fraction_1 * entropy

	return abs(attribute_entropy)


In [10]:
find_entropy_attribute(data,'Weight')

2.0611193784235506e-16

In [11]:
def find_winner(df):

	attribute_entropy = []

	IG = []

	for key in df.keys()[:-1]:
		IG.append(entropy(df.keys()[-1]) - find_entropy_attribute(df,key))

	return df.keys()[:-1][np.argmax(IG)]


In [12]:
def get_subtable(df, node, value):

	return df[df[node] == value].reset_index(drop = True)


In [13]:
def buildTree(df, tree = None):

	Target = df.keys()[-1] 

	# Get attribute with maximum information gain
	node = find_winner(df)

	#Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
	att_Value = np.unique(df[node])


	# create an empty dictionary to create tree

	if tree is None:
		tree = {}
		tree[node] = {}

	# We make loop to construct a tree by calling this function recursively. 
    # In this we check if the subset is pure and stops if it is pure. 

	for value in att_Value:
		subtable = get_subtable(df, node, value)

		cl_Value, counts = np.unique(subtable['Label'], return_counts = True)

		if len(counts) == 1:
			tree[node][value] = cl_Value[0]  # checking purity of subset

		else:
			tree[node][value] = buildTree(subtable) # calling the function recursively

	return tree

In [14]:
def predict(input_data, tree):

	# this function is used to predict for any input variable

	# recursively we go through the tree that we built

	for i in tree.values():
		tree_values = i
	
	t_values = []

	for i in tree_values:
		t_values.append(int(i)) 

	for nodes in tree.keys():
		prediction = 'Cricket'
		
		value = int(input_data[nodes])

		if value in t_values:
			tree = tree[nodes][value]
			if type(tree) is dict:
				prediction = predict(input_data, tree)
			else:
				prediction = tree
				break
		else:
			break
	return prediction

In [15]:
def train_test_split(dataset):

	training_data = dataset.iloc[:100].reset_index(drop = True)

	testing_data = dataset.iloc[100:].reset_index(drop = True)

	return training_data, testing_data

In [16]:
def test(df, tree):

	queries = data.iloc[:,:-1].to_dict(orient = "records")

	predicted = pd.DataFrame(columns = ["Predicted"])


	for i in range(len(data)):
		predicted.loc[i,"predicted"] = predict(pd.Series(queries[i]),tree)

	print('The accuracy is: ',(np.sum(predicted["predicted"] == data[df.keys()[-1]])/len(data))*100,'%')



In [18]:
training_data = train_test_split(data)[0]
testing_data = train_test_split(data)[1]

In [19]:
tree = buildTree(training_data)

In [20]:
pprint(tree)

{'Weight': {28: 'Tennis',
            29: 'Tennis',
            30: 'Tennis',
            31: 'Tennis',
            32: 'Tennis',
            33: 'Tennis',
            34: 'Tennis',
            37: 'Tennis',
            38: 'Tennis',
            39: 'Tennis',
            40: 'Tennis',
            41: 'Tennis',
            42: 'Tennis',
            43: 'Tennis',
            44: 'Tennis',
            45: 'Tennis',
            46: 'Tennis',
            47: 'Tennis',
            48: 'Tennis',
            49: 'Tennis',
            50: 'Tennis',
            51: 'Tennis',
            52: 'Tennis',
            53: 'Tennis',
            54: 'Tennis',
            55: 'Tennis',
            57: 'Tennis',
            58: 'Tennis',
            59: 'Tennis',
            60: 'Tennis',
            61: 'Tennis',
            62: 'Tennis',
            63: 'Tennis',
            64: 'Tennis',
            65: 'Tennis',
            67: 'Tennis',
            68: 'Tennis',
            69: 'Tennis',
            

In [21]:
test(testing_data, tree)

The accuracy is:  92.24806201550388 %


In [24]:
input_data = {'Weight' : 115, 'Surface': "Smooth"}
input_data = pd.Series(input_data)
prediction = predict(input_data, tree)

In [25]:
prediction

'Cricket'

In [26]:
input_data = {'Weight' : 32, 'Surface': "Rough"}
input_data = pd.Series(input_data)
prediction = predict(input_data, tree)

In [27]:
prediction

'Tennis'